In [4]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown
from thefuzz import process
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [2]:
#Downloading data
file_id_1 = '1s0zEHlvgbIr4AKruko2yFQymK_Pp1QNi'
file_id_2 = '1sTnKn_XSgZ_8uzNo8INDtmz9WjIqziYG'
file_id_3 = '1WuCNd7d4ZUJ8OQPmGtvMk_2Fasr9v0v2'
url_1 = f"https://drive.google.com/uc?id={file_id_1}"
url_2 = f"https://drive.google.com/uc?id={file_id_2}"
url_3 = f"https://drive.google.com/uc?id={file_id_3}"
output_1 = 'data_dv.csv'
output_2 = 'data_ev.csv'
output_3 = 'data_ur.csv'
gdown.download(url_1, output_1, quiet=False)
gdown.download(url_2, output_2, quiet=False)
gdown.download(url_3, output_3, quiet=False)
df1 = pd.read_csv(output_1)
df2 = pd.read_csv(output_2)
df3 = pd.read_csv(output_3,sep = ';')

# Viewing data
df1.head()
df2.head()
df3.head()

#Cleaning data and changing name col
df1['disitrict_name'] = df1['disitrict_name'].astype(str)
df2['Disitrict'] = df2['Disitrict'].astype(str)
df2['district_clean'] = df2['Disitrict'].apply(lambda x:process.extractOne(x,df1['disitrict_name'])[0])
df3.drop(columns=['Kod','Unnamed: 3'], inplace=True)
df3.rename(columns={'Nazwa':'district_name','ogółem;2023;[%]':'unemployment_rate'},inplace=True)
df3['district_clean3'] = df3['district_name'].apply(lambda x: process.extractOne(x,df1['disitrict_name'])[0])

#Merging data into one DF
df_merged = df1.merge(df2,left_on='disitrict_name',right_on='district_clean',how='inner')
df_merged = df_merged.merge(df3,left_on='disitrict_name',right_on='district_clean3', how='inner')
df_merged.head()
df_merged.rename(columns = {'disitrict_name':'district_name'},inplace=True)
df_merged.drop(columns=['Disitrict','district_clean','district_clean3'], inplace=True)
df_merged.dropna(inplace=True)
df_merged.info()
float_list = ['salary','education_expenses']
for i in float_list:
    df_merged[i] = df_merged[i].astype(str)
    df_merged[i] = df_merged[i].apply(lambda x:x.replace(',',''))
    df_merged[i] = df_merged[i].astype(float)

df_merged['unemployment_rate'] = df_merged['unemployment_rate'].str.replace(',','.')
df_merged['unemployment_rate'] = df_merged['unemployment_rate'].astype(float)
df_merged.drop_duplicates(subset = ['district_name'], keep ='first', inplace = True)
df_merged.head()


Downloading...
From: https://drive.google.com/uc?id=1s0zEHlvgbIr4AKruko2yFQymK_Pp1QNi
To: C:\Users\dawid\DataspellProjects\DAM\data_dv.csv
100%|██████████| 9.14k/9.14k [00:00<00:00, 103kB/s]
Downloading...
From: https://drive.google.com/uc?id=1sTnKn_XSgZ_8uzNo8INDtmz9WjIqziYG
To: C:\Users\dawid\DataspellProjects\DAM\data_ev.csv
100%|██████████| 17.4k/17.4k [00:00<00:00, 199kB/s]
Downloading...
From: https://drive.google.com/uc?id=1WuCNd7d4ZUJ8OQPmGtvMk_2Fasr9v0v2
To: C:\Users\dawid\DataspellProjects\DAM\data_ur.csv
100%|██████████| 13.3k/13.3k [00:00<00:00, 326kB/s]


<class 'pandas.core.frame.DataFrame'>
Index: 514 entries, 0 to 612
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   district_name       514 non-null    object 
 1   mean_score          514 non-null    float64
 2   salary              514 non-null    object 
 3   education_expenses  514 non-null    object 
 4   year                514 non-null    float64
 5   divorces            514 non-null    float64
 6   district_name       514 non-null    object 
 7   unemployment_rate   514 non-null    object 
dtypes: float64(3), object(5)
memory usage: 36.1+ KB


,district_name,mean_score,salary,education_expenses,year,divorces,district_name,unemployment_rate
0,aleksandrowski,71.580247,6019.18,857.55,2023.0,11.6,Powiat aleksandrowski,10.9
1,brodnicki,74.575000,5919.35,155.69,2023.0,11.9,Powiat brodnicki,6.7
2,bydgoski,47.770270,6461.57,635.18,2023.0,19.4,Powiat bydgoski,3.1
3,Bydgoszcz,72.275621,7478.19,566.55,2023.0,19.9,Powiat m. Bydgoszcz,2.2
4,golubsko-dobrzy?ski,78.767857,6678.54,696.36,2023.0,10.1,Powiat golubsko-dobrzyński,10.2


In [5]:
#Linear regression OLS method without scaling
X = df_merged[['salary','education_expenses','unemployment_rate','divorces']]
y = df_merged['mean_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
model_OLS = LinearRegression()
model_OLS.fit(X_train, y_train)
R2 = model_OLS.score(X_test, y_test)
df_intercept = pd.DataFrame(model_OLS.intercept_,['Intercept'],columns=['Value'])
df_coef = pd.DataFrame(model_OLS.coef_,X.columns,columns=['Value'])
df_equation = pd.concat([df_intercept, df_coef],ignore_index=False)

# Validation of model
y_pred_OLS = model_OLS.predict(X_test)
mae_OLS = mean_absolute_error(y_test, y_pred_OLS)
mse_OLS = mean_squared_error(y_test, y_pred_OLS)
r2_OLS = r2_score(y_test, y_pred_OLS)
rmse_OLS = np.sqrt(mse_OLS)

# Creating data frane of results
df_results_OLS = pd.DataFrame([mae_OLS, mse_OLS, r2_OLS, rmse_OLS], index=['MAE','MSE','R2','RMSE'],columns=['Value'])
print(df_results_OLS)

          Value
MAE    5.855805
MSE   53.003729
R2     0.041051
RMSE   7.280366


In [16]:
# Polynominal regression
poly = PolynomialFeatures(degree=2)
X_poly_train = poly.fit_transform(X_train)
model_OLS.fit(X_poly_train,y_train)
x_poly_test = poly.transform(X_test)
y_pred_poly = model_OLS.predict(x_poly_test)
mae_OLS_poly = mean_absolute_error(y_test, y_pred_poly)
R2_poly = r2_score(y_test, y_pred_poly)
mse_OLS_poly = mean_squared_error(y_test, y_pred_poly)
rmse_OLS_poly = np.sqrt(mse_OLS_poly)
df_results_OLS_Poly = pd.DataFrame([mae_OLS_poly,R2_poly,mse_OLS_poly,rmse_OLS_poly]
                                   ,index = ['MAE','R2','MSE','RMSE'],columns=['Value'])
print(df_results_OLS_Poly)


          Value
MAE    5.654987
R2     0.099199
MSE   49.789737
RMSE   7.056184
